In [1]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_validate, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix

## read data 
CIP_data_no_drop = pd.read_csv("CIP_data_encode_prev_not_dropped.csv")


In [6]:
## PI - frequency of importance 

n_iterations = 1

PI_dataframe = pd.DataFrame({'Thresold':pd.Series(dtype = 'float'),
'MSM':pd.Series(dtype = 'int'),
'MSMW':pd.Series(dtype = 'int'),
'MSW':pd.Series(dtype = 'int'),
'Other':pd.Series(dtype = 'int'),
'Midwest':pd.Series(dtype = 'int'),
'Northeast':pd.Series(dtype = 'int'),
'West':pd.Series(dtype = 'int'),
'Southeast':pd.Series(dtype = 'int'),
'Southwest':pd.Series(dtype = 'int'),
'Region':pd.Series(dtype = 'int'),
'Clinic':pd.Series(dtype = 'int')})

PI_dataframe_all = pd.DataFrame()

model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= 13, alpha = 1.291549665014884, random_state=10, learning_rate = 'constant' )

for i in range(n_iterations):
    #Generate new test and train data 
            # train
    train_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010])]
    X_train = train_data[['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC']]
    y_train = train_data['Susceptible']

    oversample = RandomOverSampler(sampling_strategy = 0.5,random_state=42)
    X_train, y_train = oversample.fit_resample(X_train,y_train)

            # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])]
    X_test = test_data[['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC']]
    y_test = test_data['Susceptible']
    oversample = RandomOverSampler(sampling_strategy = 0.5,random_state=42)
    X_test, y_test = oversample.fit_resample(X_test,y_test)

    # Train model on training data - don't need to test on test data 
    model_fit_train = model_nn.fit(X_train, y_train)

    # Permutation importance 
    PI = permutation_importance(model_fit_train, X_test, y_test, n_repeats = 10, random_state = 42)
    feature_names = ['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC']
  
    #Save results. 1 = +ve, -1 = -ve 
    PI_dataframe_all.append(PI)


KeyboardInterrupt: 